In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import requests
import json

In [2]:
#load in csv
index_us_pts = pd.read_csv("Summer-Olympic-medals-1976-to-2008.csv", encoding="latin-1")
olympics_data.head()

NameError: name 'olympics_data' is not defined

In [ ]:
# Assign Medal Counts
medal_dict = {'Gold':3,'Silver':2,'Bronze':1}
olympics_data['Points'] = olympics_data['Medal'].map(medal_dict)

In [ ]:
# Load in Host Country CSV, merge with Olympics Data Frame
host_country_df = pd.read_csv('Host_Countries.csv')
host_country_df = host_country_df[['Year','Country']]
host_country_df.rename(columns={'Country':'Host Country'},inplace=True)
olympics_data = olympics_data.merge(host_country_df,on='Year')

In [ ]:
# Get pt Total DF by year
pt_total_df = olympics_data.groupby('Year').sum()

In [ ]:
# Find Host Years and Host City, Country

host_years = olympics_data['Year'].unique()

year_countries_dict = {}
for i in host_years:
    if i == 1988.0:
        year_countries_dict[i] = 'Korea, South'
    elif i == 1980.0:
        year_countries_dict[i] = 'Soviet Union'
    else:
        year_countries_dict[i] = olympics_data[olympics_data['Year']==i]['Host Country'].unique()[0]
        
# Create DF with only host countries

host_countries = []
for k,v in year_countries_dict.items():
    host_countries.append(v)

host_df = olympics_data[olympics_data['Country'].isin(host_countries)]

# Build Data Frame with a Year index and Host Country Point Totals as Columns

frame = pd.DataFrame(index=host_years)

for year in host_years:
    country = year_countries_dict[year]

    to_add = host_df[host_df['Country']==country].groupby('Year').sum()
    to_add.rename(columns={'Points':country},level=0,inplace=True)
    frame = frame.merge(to_add,how='outer',left_index=True,right_index=True)

frame.rename(columns={'United States_x':'United States'},inplace=True)
frame = frame[['Canada','Soviet Union','United States','Korea, South','Spain','Australia','Greece','China']]
frame

In [ ]:
# Add Point Total COlumn
frame_pt = frame.merge(pt_total_df,left_index=True,right_index=True)

In [ ]:
host_country_df['City'] = olympics_data['City'].unique()
host_country_df['Search'] = host_country_df['City']+', '+host_country_df['Host Country']

In [ ]:
# Get Lat and Long

from config import g_key

city_dict = {}

search_items = []
for i in host_country_df['Search'].values:
    search_items.append(i)

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for item in search_items:
    
    params = {"address": item, "key": g_key}
    
    response = requests.get(base_url, params=params)

    geo = response.json()

    lat = geo["results"][0]["geometry"]["location"]["lat"]
    lng = geo["results"][0]["geometry"]["location"]["lng"]

    city_dict[item] = (lat,lng)

In [ ]:
host_country_df['LAT'] = host_country_df['Search'].apply(lambda x: city_dict[x][0])
host_country_df['LNG'] = host_country_df['Search'].apply(lambda x: city_dict[x][1])

In [ ]:
us_pts = [337,1,784,411,452,628,574,571,674]

In [ ]:
tot = pt_total_df.values

In [ ]:
new = zip(us_pts,tot)

In [ ]:
us_percentage = []

for i in new:
    g = i[0]
    h = i[1][0]
    new = g/h
    us_percentage.append(new)

In [ ]:
host_country_df

In [ ]:
us_pts = host_df[host_df['Country']=='United States'].groupby('Year').sum()

us_pts = [337,1,784,411,452,628,574,571,674]
av = sum(us_pts)/len(us_pts)
index_us_pts = []



index_us_pts = []
for i in us_pts:
    pt = i/av
    index_us_pts.append(pt)
    
index_us_pts  

In [ ]:
av = sum(us_pts)/len(us_pts)
av

In [ ]:
import gmaps

gmaps.configure(api_key=g_key)
locations = host_country_df[["LAT", "LNG"]]
points = frame_pt['Points'].values

# Plot Heatmap
fig = gmaps.figure()
fig.figzize=(15,5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=us_percentage,
                                 dissipating=False, max_intensity=1,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
mean_pts = [70.875,728,553.875,115,553.875,74.11111,166.666667,18,199.857143]

In [ ]:
frame.describe().T['max']

In [ ]:
x = ['Canada', 'Soviet Union', 'United States', 'Korea, South', 'Spain',
       'Australia', 'Greece', 'China']
y = [28,953,784,170,174,372,65,385]

frame_pt[frame_pt.columns[:-1]].plot(kind='box',figsize=(15,5));
plt.plot([1,2,3,4,5,6,7,8],y,'bo');

In [ ]:
frame.columns

In [ ]:
frame

In [ ]:
frame

In [ ]:
sov_union = olympics_data[olympics_data['Country']=='Soviet Union'].groupby('Year').sum()
russia = olympics_data[olympics_data['Country']=='Russia'].groupby('Year').sum()
unif_team = olympics_data[olympics_data['Country']=='Unified team'].groupby('Year').sum()

sov_union_russia = sov_union.append(russia)
sov_union_russia = sov_union_russia.append(unif_team)
sov_union_russia = sov_union_russia.sort_index()

In [ ]:
x = 0
l = sov_union_russia['Points'].values
sov_rus_pts = []

for i in range(2):
    sov_rus_pts.append(float(l[i]))
sov_rus_pts.append(np.nan)
for i in range(2,8):
    sov_rus_pts.append(float(l[i]))
sov_rus_pts


In [ ]:
frame['Soviet Union/Russia'] = sov_rus_pts

In [ ]:
frame = frame[['Canada', 'Soviet Union/Russia', 'United States', 'Korea, South', 'Spain',
       'Australia', 'Greece', 'China']]

In [ ]:
frame

In [ ]:
x = ['Canada', 'Soviet Union/Russia', 'United States', 'Korea, South', 'Spain',
       'Australia', 'Greece', 'China']
y = [28,953,784,628,170,174,372,65,385]

frame[frame.columns].plot(kind='box',figsize=(15,5))
plt.title('Medal Point Performance Box Plot Highlighted by Host Year')
plt.ylabel('Medal Points Awarded')
# plt.xlabel('Host Countries from 1976 - 2008')
plt.plot([1,2,3,3,4,5,6,7,8],y,'bo');

plt.legend(labels='O'[0],title='Year Hosted');


In [ ]:
frame

In [ ]:
frame.columns

In [ ]:
frame[frame.columns]

In [ ]:
frame['Canada'].mean()

In [ ]:
frame.columns

In [ ]:
new_dict = {'Canada':[], 'Soviet Union/Russia':[], 'United States':[], 'Korea, South':[], 'Spain':[],
       'Australia':[], 'Greece':[], 'China':[]}

for col in frame.columns:
    avg = frame[col].mean()
    for line in frame[col]:
        new_dict[col].append(line/avg)
new_dict

In [ ]:
pd.DataFrame(new_dict).plot(kind='box',figsize=(15,5))

In [ ]:
host_year = {1976:'Canada',1980:'Soviet Union/Russia',1984:'United States',1988:'Korea, South',1992:'Spain',1996:'United States',
             2000:'Australia',2004:'Greece', 2008:'China'}

for country in ['Canada', 'Soviet Union/Russia', 'United States', 'Korea, South',
       'Spain', 'Australia', 'Greece', 'China']:
    frame[country]
frame
    

In [ ]:
frame['Canada'].loc(0)

In [ ]:
frame['Canada'][0:]

In [ ]:
frame.columns